In [19]:
from datetime import datetime
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import talib as ta

In [20]:
# Ruta credenciales de CGP
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_data'
dataset = 'bronze'

# Tables id
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_indicators'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [21]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [22]:
# Solamente nos traemos informacion del SP500
df_all = bigquery.run_query(
    f"""
    SELECT
        ti.*
    FROM {table_conca} AS ti
    INNER JOIN `bronze.bronze_sp500_tickers` AS sp
        ON sp.ticker = ti.ticker
    """
)
df_all

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-26 00:00:00+00:00,53.820000,55.044998,53.779999,54.750000,54.750000,12155181,CMG,5defa416d05e811292a5a77f7c217116
1,2024-08-26 00:00:00+00:00,132.669998,133.754501,131.750000,132.500000,132.500000,919298,NTAP,6063f2fac611afcb6a7de6db4fbd5f18
2,2024-08-26 00:00:00+00:00,148.970001,150.089996,147.550003,148.500000,148.500000,5778700,CVX,6ab95de308f82f5a88d667bdbf4a3423
3,2024-08-26 00:00:00+00:00,58.240002,58.490002,58.040001,58.220001,58.220001,683600,LNT,22a40b2ff11abb14ca7482ad7bd73707
4,2024-08-26 00:00:00+00:00,181.479996,183.270004,179.190002,182.130005,182.130005,2082791,GEV,37ca527aee118b835ac4d870eff52d9b
...,...,...,...,...,...,...,...,...,...
1191492,2024-08-19 00:00:00+00:00,34.910000,36.700001,34.910000,36.580002,36.580002,3884700,MTCH,cd0484f31059c8724b858370c1dc9b8f
1191493,2024-08-20 00:00:00+00:00,36.360001,36.595001,36.169998,36.529999,36.529999,3375900,MTCH,d7b903d7ef9849e21d50e60813dbfebd
1191494,2024-08-21 00:00:00+00:00,36.689999,37.580002,36.439999,37.139999,37.139999,4550400,MTCH,c2e2f3489b2be3381ced46016e4ed21d
1191495,2024-08-22 00:00:00+00:00,37.099998,37.160000,36.360001,36.419998,36.419998,4800500,MTCH,7ff4851947e5aec2c4f32365c70f10a3


In [23]:
df_drop_nan = df_all.dropna()
df_drop_nan

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-26 00:00:00+00:00,53.820000,55.044998,53.779999,54.750000,54.750000,12155181,CMG,5defa416d05e811292a5a77f7c217116
1,2024-08-26 00:00:00+00:00,132.669998,133.754501,131.750000,132.500000,132.500000,919298,NTAP,6063f2fac611afcb6a7de6db4fbd5f18
2,2024-08-26 00:00:00+00:00,148.970001,150.089996,147.550003,148.500000,148.500000,5778700,CVX,6ab95de308f82f5a88d667bdbf4a3423
3,2024-08-26 00:00:00+00:00,58.240002,58.490002,58.040001,58.220001,58.220001,683600,LNT,22a40b2ff11abb14ca7482ad7bd73707
4,2024-08-26 00:00:00+00:00,181.479996,183.270004,179.190002,182.130005,182.130005,2082791,GEV,37ca527aee118b835ac4d870eff52d9b
...,...,...,...,...,...,...,...,...,...
1191492,2024-08-19 00:00:00+00:00,34.910000,36.700001,34.910000,36.580002,36.580002,3884700,MTCH,cd0484f31059c8724b858370c1dc9b8f
1191493,2024-08-20 00:00:00+00:00,36.360001,36.595001,36.169998,36.529999,36.529999,3375900,MTCH,d7b903d7ef9849e21d50e60813dbfebd
1191494,2024-08-21 00:00:00+00:00,36.689999,37.580002,36.439999,37.139999,37.139999,4550400,MTCH,c2e2f3489b2be3381ced46016e4ed21d
1191495,2024-08-22 00:00:00+00:00,37.099998,37.160000,36.360001,36.419998,36.419998,4800500,MTCH,7ff4851947e5aec2c4f32365c70f10a3


In [26]:
# Vamos a filtrar solamente aquellos tickers que tengamos más de 9 años de histórico

# Obtener el año actual
año_actual = datetime.now().year

# Agrupar por el campo 'ticker' y filtrar aquellos cuyo valor máximo de 'date' sea igual al año actual
tickers_filtrados = df_drop_nan.groupby('ticker').filter(lambda x: x['date'].max().year == año_actual)

# Filtramos aquellos tickers que tengan más de 5 años
tickers_filtrados = tickers_filtrados.groupby('ticker').count()
tickers_filtrados = tickers_filtrados[tickers_filtrados['date'] >= 252 * 9]

tickers_filtrados = tickers_filtrados.reset_index()
tickers_filtrados = tickers_filtrados['ticker']

tickers_filtrados


0         A
1       AAL
2      AAPL
3      ABBV
4       ABT
       ... 
472     XYL
473     YUM
474     ZBH
475    ZBRA
476     ZTS
Name: ticker, Length: 477, dtype: object

In [27]:
# Filtramos el df con los tickers que tienen mas de 9 años
df = pd.merge(df_drop_nan, tickers_filtrados, how='inner', on='ticker')
df = df[df.columns]
df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-26 00:00:00+00:00,53.820000,55.044998,53.779999,54.750000,54.750000,12155181,CMG,5defa416d05e811292a5a77f7c217116
1,2015-01-02 00:00:00+00:00,13.720000,13.749400,13.420200,13.568000,13.568000,16240000,CMG,3eb1868dd45d911f7cd5232dc3c91d6b
2,2015-01-05 00:00:00+00:00,13.568000,13.598400,13.284000,13.353800,13.353800,18720000,CMG,b9d8dab2864c559421e94bb1efe14507
3,2015-01-06 00:00:00+00:00,13.335800,13.440000,13.075400,13.287600,13.287600,30470000,CMG,b039e55c8038476349440d3d2b65c59f
4,2015-01-07 00:00:00+00:00,13.368000,13.897000,13.368000,13.885200,13.885200,34280000,CMG,854d74d9a4bee0829e27bf68d3c81a8e
...,...,...,...,...,...,...,...,...,...
1159634,2024-08-22 00:00:00+00:00,119.019997,121.059998,118.739998,119.000000,119.000000,7843800,TJX,e9e62bdc4b118ac18a95c765db298820
1159635,2024-08-23 00:00:00+00:00,120.129997,120.540001,118.470001,119.470001,119.470001,4522500,TJX,29c4b977676710616d3e7caf4fc038ab
1159636,2024-08-27 00:00:00+00:00,119.489998,120.529999,119.190002,120.059998,120.059998,4680000,TJX,7024a9c6e3c04df1d418e4b30ba95b0c
1159637,2024-08-28 00:00:00+00:00,120.500000,121.129997,119.129997,119.279999,119.279999,3906100,TJX,8d5c8f149f2c10ed47b277b63df76415


In [28]:
# Renombrar las columnas a letra capitalizada
df = df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume',
    'ticker': 'Ticker'
})


In [29]:
def calculate_indicators(df):
    # Indicadores técnicos
    df['SMA_20'] = ta.SMA(df['Close'], timeperiod=20)
    df['EMA_50'] = ta.EMA(df['Close'], timeperiod=50)
    df['ADX'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

    # Patrones de velas
    df['CDL_DOJI'] = ta.CDLDOJI(df['Open'], df['High'], df['Low'], df['Close'])
    df['CDL_HAMMER'] = ta.CDLHAMMER(
        df['Open'], df['High'], df['Low'], df['Close'])
    df['CDL_MORNING_STAR'] = ta.CDLMORNINGSTAR(
        df['Open'], df['High'], df['Low'], df['Close'])
    df['CDL_ENGULFING'] = ta.CDLENGULFING(
        df['Open'], df['High'], df['Low'], df['Close'])
    df['CDL_LONGLINE'] = ta.CDLLONGLINE(
        df['Open'], df['High'], df['Low'], df['Close'])

    # Otros indicadores adicionales para medir la volatilidad
    df['bb_bbh'], df['bb_bbm'], df['bb_bbl'] = ta.BBANDS(df['Close'], timeperiod=20,
                   nbdevup=2, nbdevdn=2, matype=0)
    df['daily_return'] = df['Close'].pct_change()
    df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['Close'], slowperiod=26,
                   fastperiod=12, signalperiod=9)
    df['stoch'], df['stoch_d'] = ta.STOCH(
        high=df['High'], low=df['Low'], close=df['Close'], fastk_period=21, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    # Añadimos la variable a predecir y otras métricas
    df['Tomorrow Close'] = df['Close'].shift(-1)
    df['Target'] = np.where(df['Tomorrow Close'] > df['Close'], 1, 0)
    df = df.drop(['Tomorrow Close'], axis=1)
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Volatility'] = df['Log_Return'].rolling(window=5).std() # Volatilidad a 5 días
    df['Volume_Change'] = df['Volume'].pct_change() 
    
    return df

In [30]:
df = df.groupby('Ticker', group_keys=False).apply(calculate_indicators)

# Mostrar el DataFrame final
df.head()

,date,Open,High,Low,Close,adjclose,Volume,Ticker,id,SMA_20,...,daily_return,macd,macd_signal,macd_hist,stoch,stoch_d,Target,Log_Return,Volatility,Volume_Change
0,2024-08-26 00:00:00+00:00,53.8200,55.044998,53.779999,54.7500,54.7500,12155181,CMG,5defa416d05e811292a5a77f7c217116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,<NA>
1,2015-01-02 00:00:00+00:00,13.7200,13.749400,13.420200,13.5680,13.5680,16240000,CMG,3eb1868dd45d911f7cd5232dc3c91d6b,NaN,...,-0.752183,NaN,NaN,NaN,NaN,NaN,0,-1.395063,NaN,0.336056
2,2015-01-05 00:00:00+00:00,13.5680,13.598400,13.284000,13.3538,13.3538,18720000,CMG,b9d8dab2864c559421e94bb1efe14507,NaN,...,-0.015787,NaN,NaN,NaN,NaN,NaN,0,-0.015913,NaN,0.152709
3,2015-01-06 00:00:00+00:00,13.3358,13.440000,13.075400,13.2876,13.2876,30470000,CMG,b039e55c8038476349440d3d2b65c59f,NaN,...,-0.004957,NaN,NaN,NaN,NaN,NaN,1,-0.004970,NaN,0.627671
4,2015-01-07 00:00:00+00:00,13.3680,13.897000,13.368000,13.8852,13.8852,34280000,CMG,854d74d9a4bee0829e27bf68d3c81a8e,NaN,...,0.044974,NaN,NaN,NaN,NaN,NaN,1,0.043992,NaN,0.125041


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1159639 entries, 0 to 1159638
Data columns (total 31 columns):
 #   Column            Non-Null Count    Dtype              
---  ------            --------------    -----              
 0   date              1159639 non-null  datetime64[us, UTC]
 1   Open              1159639 non-null  float64            
 2   High              1159639 non-null  float64            
 3   Low               1159639 non-null  float64            
 4   Close             1159639 non-null  float64            
 5   adjclose          1159639 non-null  float64            
 6   Volume            1159639 non-null  Int64              
 7   Ticker            1159639 non-null  object             
 8   id                1159639 non-null  object             
 9   SMA_20            1150576 non-null  float64            
 10  EMA_50            1136266 non-null  float64            
 11  ADX               1146760 non-null  float64            
 12  RSI               1152961 non-nul

In [32]:
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                    0
Open                    0
High                    0
Low                     0
Close                   0
adjclose                0
Volume                  0
Ticker                  0
id                      0
SMA_20               9063
EMA_50              23373
ADX                 12879
RSI                  6678
CDL_DOJI                0
CDL_HAMMER              0
CDL_MORNING_STAR        0
CDL_ENGULFING           0
CDL_LONGLINE            0
bb_bbh               9063
bb_bbm               9063
bb_bbl               9063
daily_return          477
macd                15741
macd_signal         15741
macd_hist           15741
stoch               11448
stoch_d             11448
Target                  0
Log_Return            477
Volatility           2385
Volume_Change         477
dtype: int64

In [33]:
df = df.dropna()
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                0
Open                0
High                0
Low                 0
Close               0
adjclose            0
Volume              0
Ticker              0
id                  0
SMA_20              0
EMA_50              0
ADX                 0
RSI                 0
CDL_DOJI            0
CDL_HAMMER          0
CDL_MORNING_STAR    0
CDL_ENGULFING       0
CDL_LONGLINE        0
bb_bbh              0
bb_bbm              0
bb_bbl              0
daily_return        0
macd                0
macd_signal         0
macd_hist           0
stoch               0
stoch_d             0
Target              0
Log_Return          0
Volatility          0
Volume_Change       0
dtype: int64

In [14]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca_to_save, new_df=df)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)
    print('New data persisted')
    print(f'Exception encountered: {e}')

No new records to load.
